In [1]:
import numpy as np
import pandas as pd
from river import optim
from river import metrics
from river.drift import EDDM
from sail.models.auto_ml.tune import SAILTuneGridSearchCV
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.models.river.linear_model import LinearRegression
from sail.models.auto_ml.auto_pipeline import SAILAutoPipeline
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sail.drift_detection.drift_detector import SAILDriftDetector
from sail.transfomers.river.preprocessing import StandardScaler

#### Load Data


In [2]:
X = pd.read_csv("../../datasets/HDWF2.csv")

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [3]:
linear_reg = LinearRegression(optimizer=optim.SGD(0.1))
random_forest = AdaptiveRandomForestRegressor()

#### Create SAIL Pipeline


In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("regressor", "passthrough"),
]
sail_pipeline = SAILPipeline(steps=steps, scoring=metrics.R2)

#### HPO Params


In [5]:
params_grid = [
    {
        "regressor": [linear_reg],
        "regressor__l2": [0.1, 0.9],
        "regressor__intercept_init": [0.2, 0.5],
    },
    {
        "regressor": [random_forest],
        "regressor__n_models": [10, 15, 20]
    },
]

#### Create Model Instance


In [6]:
auto_pipeline = SAILAutoPipeline(
    pipeline=sail_pipeline,
    pipeline_params_grid=params_grid,
    search_method=SAILTuneGridSearchCV,
    search_method_params={
        "num_cpus_per_trial": 1,
        "max_iters": 1,
        "early_stopping": False,
        "mode": "max",
        "scoring": "r2",
        "pipeline_auto_early_stop": False,
        "keep_best_configurations": 2
    },
    search_data_size=1000,
    incremental_training=True,
    drift_detector=SAILDriftDetector(model=EDDM(), drift_param="difference"),
    pipeline_strategy="DetectAndIncrement",
)

[2023-07-11 21:17:43:175] - INFO - SAIL : Pipeline Strategy [DetectAndIncrement] created with actions: ['DATA_COLLECTION', 'FIND_BEST_PIPELINE', 'SCORE_AND_DETECT_DRIFT', 'PARTIAL_FIT_MODEL']


#### Start Training


In [7]:
y_preds = []
y_true = []
batch_size = 50

for start in range(0, 1500, batch_size):

    end = start + batch_size

    X_train = X.iloc[start:end]
    y_train = y.iloc[start:end]

    if end > 1000: # search_data_size is 1000
        preds = auto_pipeline.predict(X_train)
        y_preds.extend(list(preds))
        y_true.extend(list(y_train))

    auto_pipeline.train(X_train, y_train)

2023-07-11 21:18:03,973	INFO tune.py:945 -- Total run time: 17.11 seconds (17.10 seconds for the tuning loop).
SAIL Pipeline Score: 100%========== [Points: 1000/1000, Elapsed:0.0051s, Metric=R2, Batch Size=1000, Score=-1.84]  
SAIL Pipeline Score: 100%========== [Points: 1000/1000, Elapsed:0.0048s, Metric=R2, Batch Size=1000, Score=0.843]
SAIL Pipeline fit: 100%========== [Steps: 3/3, ETA: 00:03<00:00, Elapsed:3.018s, Batch Size=1000, Score=0.843]            
[2023-07-11 21:18:09:282] - INFO - SAIL : Pipeline tuning completed. Disconnecting Ray cluster...
[2023-07-11 21:18:09:282] - INFO - SAIL : Found best params: {'regressor': AdaptiveRandomForestRegressor(n_models=20), 'regressor__n_models': 20}


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0014s, Metric=R2, Batch Size=50, P_Score=0.595]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0010s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.161s, Batch Size=50, P_Score=0.595]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0012s, Metric=R2, Batch Size=50, P_Score=0.751]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0009s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.159s, Batch Size=50, P_Score=0.751]            


...

SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0012s, Metric=R2, Batch Size=50, P_Score=0.655]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0007s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.163s, Batch Size=50, P_Score=0.655]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0013s, Metric=R2, Batch Size=50, P_Score=0.681]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0010s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.162s, Batch Size=50, P_Score=0.681]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0027s, Metric=R2, Batch Size=50, P_Score=0.682]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0011s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.180s, Batch Size=50, P_Score=0.682]            


#### Save Model


In [8]:
auto_pipeline.save_model(".")

[2023-07-11 21:18:11:282] - INFO - SAIL (SAILAutoPipeline) : making directory tree ./sail_auto_pipeline
[2023-07-11 21:18:13:109] - INFO - SAIL (SAILPipeline) : making directory tree ./sail_auto_pipeline/pipeline_strategy/best_pipeline


'./sail_auto_pipeline'

#### Load Model


In [9]:
new_auto_pipeline = SAILAutoPipeline.load_model(".")

[2023-07-11 21:18:14:888] - INFO - SAIL : Pipeline Strategy [DetectAndIncrement] created with actions: ['DATA_COLLECTION', 'FIND_BEST_PIPELINE', 'SCORE_AND_DETECT_DRIFT', 'PARTIAL_FIT_MODEL']
SAIL Pipeline Progressive Score: 100%========== [Points: 1000/1000, Elapsed:0.0028s, Metric=R2, Batch Size=1000, P_Score=0.682]


#### Continue training using the load model


In [10]:
for start in range(1501, 2001, batch_size):

    end = start + batch_size

    X_train = X.iloc[start:end]
    y_train = y.iloc[start:end]

    if end > 1000: # search_data_size is 1000
        preds = new_auto_pipeline.predict(X_train)
        if preds is not None:
            y_preds.extend(list(preds))
            y_true.extend(list(y_train))

    new_auto_pipeline.train(X_train, y_train)

>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0047s, Metric=R2, Batch Size=50, P_Score=0.689]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0040s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.151s, Batch Size=50, P_Score=0.689]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0011s, Metric=R2, Batch Size=50, P_Score=0.687]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0008s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.147s, Batch Size=50, P_Score=0.687]            


>>>--------------------------------------------------------------------------------------------


...

SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0014s, Metric=R2, Batch Size=50, P_Score=0.717]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0010s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.179s, Batch Size=50, P_Score=0.717]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0015s, Metric=R2, Batch Size=50, P_Score=0.714]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0013s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.194s, Batch Size=50, P_Score=0.714]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Progressive Score: 100%========== [Points: 50/50, Elapsed:0.0021s, Metric=R2, Batch Size=50, P_Score=0.721]
SAIL Drift detection: 100%========== [Points: 50/50, Elapsed:0.0017s, Detector=EDDM, Batch Size=50, Param=differene, Drift=No]
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.204s, Batch Size=50, P_Score=0.721]            


#### Final Score


In [11]:
new_auto_pipeline.progressive_score

0.7211272266954489

### Plot predictions


In [12]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds}).head(500)
fig = px.line(df, y=["y_true", "y_preds"], title='')
fig.show()